# Data Cleaning

### Importing 

In [2]:
import pandas as pd #Mainly for dataFrame.
import numpy as np #For numeric python like nD arrays.
import matplotlib.pyplot as plt #for plotting graph.
import language_tool_python #for grammer checking.
from spellchecker import SpellChecker #For spell checking.
import multiprocessing as mp #To process data fast.
from sklearn.preprocessing import MinMaxScaler #Scale Score data in 1-100 range.
import re

### Reading dataset

In [2]:
pd.options.display.max_colwidth = 10000
df = pd.read_excel("dataset.xlsx")
df.info()
'''
    essay_id: A unique identifier for each individual student essay
    essay_set: 1-8, an id for each set of essays
    essay: The ascii text of a student's response
    rater1_domain1: Rater 1's domain 1 score; all essays have this #score out of 4-30
    rater2_domain1: Rater 2's domain 1 score; all essays have this #score out of 4-30
    rater3_domain1: Rater 3's domain 1 score; only some essays in set 8 have this.
    domain1_score: Resolved score between the raters; all essays have this #score out of 8-60
    rater1_domain2: Rater 1's domain 2 score; only essays in set 2 have this #out of 4
    rater2_domain2: Rater 2's domain 2 score; only essays in set 2 have this #out of 4
    domain2_score: Resolved score between the raters; only essays in set 2 have this #out of 4
    rater1_trait1 score - rater3_trait6 score: trait scores for sets 7-8
    rater1_trait1-rater1_trait4 have all 7 and 8 set rated 
    rater2_trait1-rater2_trait4 have all 7 and 8 set rated   
'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12978 entries, 0 to 12977
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   essay_id        12978 non-null  int64  
 1   essay_set       12978 non-null  int64  
 2   essay           12978 non-null  object 
 3   rater1_domain1  12977 non-null  float64
 4   rater2_domain1  12977 non-null  float64
 5   rater3_domain1  128 non-null    float64
 6   domain1_score   12977 non-null  float64
 7   rater1_domain2  1800 non-null   float64
 8   rater2_domain2  1800 non-null   float64
 9   domain2_score   1800 non-null   float64
 10  rater1_trait1   2292 non-null   float64
 11  rater1_trait2   2292 non-null   float64
 12  rater1_trait3   2292 non-null   float64
 13  rater1_trait4   2292 non-null   float64
 14  rater1_trait5   723 non-null    float64
 15  rater1_trait6   723 non-null    float64
 16  rater2_trait1   2292 non-null   float64
 17  rater2_trait2   2292 non-null  

"\n    essay_id: A unique identifier for each individual student essay\n    essay_set: 1-8, an id for each set of essays\n    essay: The ascii text of a student's response\n    rater1_domain1: Rater 1's domain 1 score; all essays have this #score out of 4-30\n    rater2_domain1: Rater 2's domain 1 score; all essays have this #score out of 4-30\n    rater3_domain1: Rater 3's domain 1 score; only some essays in set 8 have this.\n    domain1_score: Resolved score between the raters; all essays have this #score out of 8-60\n    rater1_domain2: Rater 1's domain 2 score; only essays in set 2 have this #out of 4\n    rater2_domain2: Rater 2's domain 2 score; only essays in set 2 have this #out of 4\n    domain2_score: Resolved score between the raters; only essays in set 2 have this #out of 4\n    rater1_trait1 score - rater3_trait6 score: trait scores for sets 7-8\n    rater1_trait1-rater1_trait4 have all 7 and 8 set rated \n    rater2_trait1-rater2_trait4 have all 7 and 8 set rated   \n"

In [11]:
scaler = MinMaxScaler(feature_range=(1,100))
#Hence they are not depend on any other set so we can consider full column
df["domain2_score"] = scaler.fit_transform(df.domain2_score.values.reshape(-1, 1))
df["rater1_trait1"] = scaler.fit_transform(df.rater1_trait1.values.reshape(-1, 1))
df["rater1_trait2"] = scaler.fit_transform(df.rater1_trait2.values.reshape(-1, 1))
df["rater1_trait3"] = scaler.fit_transform(df.rater1_trait3.values.reshape(-1, 1))
df["rater1_trait4"] = scaler.fit_transform(df.rater1_trait4.values.reshape(-1, 1))
df["rater2_trait1"] = scaler.fit_transform(df.rater2_trait1.values.reshape(-1, 1))
df["rater2_trait2"] = scaler.fit_transform(df.rater2_trait2.values.reshape(-1, 1))
df["rater2_trait3"] = scaler.fit_transform(df.rater2_trait3.values.reshape(-1, 1))
df["rater2_trait4"] = scaler.fit_transform(df.rater2_trait4.values.reshape(-1, 1))

#Domain1_Score is given for all eassy and for different set have different max score.
df1 = pd.DataFrame()
for i in range(1,9):
    temp = pd.DataFrame(df[df["essay_set"]==i])
    temp["domain1_score"] = scaler.fit_transform(temp.domain1_score.values.reshape(-1, 1))
    if i==1:
        df1 = temp
    else:
        df1 = df1.append(temp, ignore_index = True) 

In [12]:
newCol = ["essay_id","essay_set","essay","domain1_score","domain2_score","rater1_trait1","rater1_trait2","rater1_trait3","rater1_trait4","rater2_trait1","rater2_trait2","rater2_trait3","rater2_trait4"]
DS = df1[newCol]
ncol = list(newCol)
ncol.append("Total_Score")
newDf = pd.DataFrame()
for i in range(1,9):
    temp = DS[DS["essay_set"]==i];
    if i in [1,3,4,5,6]:
        temp["Total_Score"] = temp["domain1_score"]
    else:
        if i==2:
            temp["Total_Score"] = (temp["domain1_score"]+temp["domain2_score"])/2
        else:
            ls = [temp["rater1_trait1"],temp["rater1_trait2"],temp["rater1_trait3"],temp["rater1_trait4"],temp["rater2_trait1"],temp["rater2_trait2"],temp["rater2_trait3"],temp["rater2_trait4"]]
            temp["Total_Score"] = sum(ls)/8
    if i==1:
        newDf = temp
    else:
        newDf = newDf.append(temp, ignore_index = True) 


<ipython-input-12-87a75b3f89e9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["Total_Score"] = temp["domain1_score"]
<ipython-input-12-87a75b3f89e9>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["Total_Score"] = (temp["domain1_score"]+temp["domain2_score"])/2
<ipython-input-12-87a75b3f89e9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [13]:
# spellMistake = []
# wordCount = []
# sentenceCount = []
# grammerMistake = []
# spell = SpellChecker()
# for i in range(len(newDf)):
#     ls = set(newDf["essay"][i].split())
#     misspelled = spell.unknown(ls)
#     spellMistake.append(len(misspelled))
    
#     wordCount.append(len(ls))
#     #Number of Unique words;
#     sentenceCount.append(len(re.split("\.\s+", newDf["essay"][i]))) 
#     #Will count sentence as dot-space split;
  
#     tool = language_tool_python.LanguageToolPublicAPI('en-US')
#     error = tool.check(newDf["essay"][i])
#     grammerMistake.append(len(error))


In [14]:
# saveDf = newDf
# saveDf["wordCount"] = wordCount
# saveDf["sentenceCount"] = sentenceCount
# saveDf["spellMistake"] = spellMistake
# saveDf["grammerMistake"] = grammerMistake
# saveDf.to_csv("name.csv")

# Data Cleaning for NLP


In [52]:
import re  
# Natural Language Tool Kit 
import nltk  
# to remove stopword 
from nltk.corpus import stopwords 
# for Stemming propose  
from nltk.stem.porter import PorterStemmer 
# Initialize empty array 
# to append clean text  
corpus = []  

word = pd.read_excel("word_contraction.xlsx")
ls1 = list(word["C1"])
ls2 = list(word["C2"])
df = pd.read_csv("Processed_DataSet.csv")
score = list(df["Total_Score"])
eassy = list(df["essay"])

# Replaceing had'nt with had not
for i in range(len(eassy)):
    s = eassy[i];
    for j in range(len(ls1)):
        s = s.replace(ls1[j],ls2[j])
    eassy[i] = s;


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shinchan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
for i in range(len(eassy)):  
      
    # column : "Review", row ith 
    review = re.sub('[^a-zA-Z]', ' ', eassy[i])  
      
    # convert all cases to lower cases 
    review = review.lower()  
      
    # split to array(default delimiter is " ") 
    review = review.split()  
      
    # creating PorterStemmer object to 
    # take main stem of each word 
    ps = PorterStemmer()  
      
    # loop for stemming each word 
    # in string array at ith row     
    review = [ps.stem(word) for word in review 
                if not word in set(stopwords.words('english'))]  
                  
    # rejoin all string array elements 
    # to create back into a string 
    review = ' '.join(review)   
      
    # append each string to create 
    # array of clean text  
    corpus.append(review)  
newData = pd.DataFrame({"Eassy":corpus, "Score":score})
# newData.to_csv("new_Data.csv")